In [1]:
from itertools import *
import numpy as np
import triqs.utility.mpi as mpi
from h5 import *
from triqs.gf import *
from triqs_dft_tools.sumk_dft import *
from triqs_dft_tools.sumk_dft_tools import *
from triqs.operators.util.hamiltonians import *
from triqs.operators.util.U_matrix import *
from triqs_cthyb import *
import warnings
from triqs.plot.mpl_interface import oplot, plt

warnings.filterwarnings("ignore", category=FutureWarning)

Starting serial run at: 2024-09-23 12:25:38.038855


In [2]:
h_field = 0.0
beta = 5
filename = f"nsp"
mesh = MeshImFreq(beta=beta, S="Fermion", n_iw=1025)

SK = SumkDFTTools(hdf_file=filename + ".h5", use_dft_blocks=False, mesh=mesh)
norb = 20

# We analyze the block structure of the Hamiltonian
Sigma = SK.block_structure.create_gf(mesh=mesh)

SK.put_Sigma([Sigma])

# Setup CTQMC Solver
n_orb = SK.corr_shells[0]["dim"]
spin_names = ["up", "down"]

# Print some information on the master node
Sigma_iw = 0
block_structure = None
if mpi.is_master_node():
    ar = HDFArchive(filename + ".h5", "a")
    if not "DMFT_results" in ar:
        ar.create_group("DMFT_results")
    if not "Iterations" in ar["DMFT_results"]:
        ar["DMFT_results"].create_group("Iterations")
    if "DMFT_input" in ar:
        block_structure = ar["DMFT_input"]["sumk_block_structure"]
    if "iteration_count" in ar["DMFT_results"]:
        iteration_offset = ar["DMFT_results"]["iteration_count"] + 1
        print(("offset", iteration_offset))
        Sigma_iw = ar["DMFT_results"]["Iterations"][
            "Sigma_it" + str(iteration_offset - 1)
        ]
        SK.dc_imp = ar["DMFT_results"]["Iterations"][
            "dc_imp" + str(iteration_offset - 1)
        ]
        SK.dc_energ = ar["DMFT_results"]["Iterations"][
            "dc_energ" + str(iteration_offset - 1)
        ]
        SK.chemical_potential = ar["DMFT_results"]["Iterations"][
            "chemical_potential" + str(iteration_offset - 1)
        ].real


if block_structure:
    SK.block_structure = block_structure
else:
    G = SK.extract_G_loc(transform_to_solver_blocks=False)
    SK.analyse_block_structure_from_gf(G, threshold=1e-3)

SK.put_Sigma(Sigma_imp=[Sigma_iw])

ikarray = numpy.array(list(range(SK.n_k)))

gf_csc = Gf(mesh=SK.mesh, target_shape=(norb, norb))
G_latt_orb = BlockGf(
    name_list=["up", "down"], block_list=[gf_csc, gf_csc], make_copies=True
)


for ik in mpi.slice_array(ikarray):
    G_latt_KS = SK.lattice_gf(ik=ik) * SK.bz_weights[ik]
    for bname, gf in G_latt_orb:
        gf += G_latt_KS[bname]
        # gf += SK.downfold(ik, 0, bname, G_latt_KS[bname], gf, shells="all", ir=0)

('offset', 10)


In [6]:
G_latt_orb

Green Function G composed of 2 blocks: 
 Greens Function G_up with mesh Imaginary Freq Mesh with beta = 5, statistic = Fermion, n_iw = 1025, positive_only = false and target_shape (20, 20): 
 
 Greens Function G_down with mesh Imaginary Freq Mesh with beta = 5, statistic = Fermion, n_iw = 1025, positive_only = false and target_shape (20, 20): 
 

In [9]:
np.sum(G_latt_orb.density()["up"]) + np.sum(G_latt_orb.density()["down"])

(15.665849444428188-2.074591628590981e-13j)

In [11]:
np.sum(np.diag(G_latt_orb.density()["up"])) + np.sum(
    np.diag(G_latt_orb.density()["down"])
)

(16.000377885010757-2.074591632315491e-13j)

In [12]:
np.sum(np.diag(G_latt_orb.density()["up"]))

(8.093119526095704-1.0006836318412225e-13j)